## Scrape cars on sale on various trader websites

Approach is different for this one. All of the trader's car can be listed on one page. I've simply loaded the entire page and saved it, with picture files. This is what I'm parsing. 

This could be set up to be done on a regular basis to identify new cars, sold cars, and so on. 

First saved file on 09/12/2016.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import sqlite3
import time

%load_ext sql
%config SqlMagic.autopandas=True
%matplotlib inline

/usr/local/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/usr/local/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [10]:
%qtconsole

In [23]:
# Connect to dbs
conn = sqlite3.connect('data/sales.db')


In [9]:
source = 'www.vroom.com'
trader = 'www.vroom.com/catalog'
# browser = webdriver.Chrome()
# browser.get(trader)

In [27]:
#Parse info
all_cars = pd.DataFrame()
date = '2016-09-12'

soup = BeautifulSoup(open('./data/VR_scrape/VR09122016.html'),"lxml")
cars = soup.find_all("div", class_="car-new")

for car in cars:
    try:
        name = car.find_all("div", class_="info")[0].find_all("div", class_="name")[0].text.split()
        year,make,model = name[0],name[1],name[2]
        trim = car.find_all("div", class_="trim")[0].text
        mileage = int(car.find_all("div", class_="mileage")[0].text.replace(',','').replace(' Miles',''))
        price = int(car.find_all("div", class_="price")[0].text.replace('$','').replace(',',''))
        img_file = car.find_all("img")[0]['src']
        all_cars = all_cars.append(pd.DataFrame(
            [[date, 'Used',year, make,model,trim,mileage,price,img_file,source]]),
                                      ignore_index = True)
    except:
        pass
    
        

In [29]:
# Add to sqlite table salesVR

all_cars.columns = ['date','new_used', 'year', 'make', 'model','trim', 'mileage', 'price', 'img_file', 'source']    
index_max = conn.execute('SELECT MAX(line_id) FROM salesVR').fetchall()
if index_max[0][0] is not None:
    all_cars.index = all_cars.index + 1 + index_max[0][0]
all_cars.to_sql('salesVR',conn,flavor='sqlite',if_exists='append',index_label='line_id')